## Install the necessary libraries

In [ ]:
!pip install openai -q
!pip install elevenlabs -q
!pip install moviepy -q

In [ ]:
from IPython.display import display, Image, Audio
import cv2
import base64
from openai import OpenAI

## Read the video and capture each frame

In [ ]:
video = cv2.VideoCapture("Bicepcurl.mp4")

base64Frames = []
while video.isOpened():
    success, frame = video.read()
    if not success:
        break
    _, buffer = cv2.imencode(".jpg", frame)
    base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

video.release()
print(len(base64Frames), "frames read.")

42 frames read.


## Prompt Engineering

In [ ]:
PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            "These are frames of a video. Create a very short voiceover script in the style of a super excited narrator similar to Morgan Freeman. Except he's narrating the point of view of a human doing mundane activities but he is still super excited.",
            *map(lambda x: {"image": x, "resize": 768}, base64Frames[0::10]),
        ],
    },
]
params = {
    "model": "gpt-4-vision-preview",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 500,
}

client = OpenAI(
    # This is the default and can be omitted
    api_key=''
)

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)

[Excited, warm intonation, a slight chuckle in the voice]
"Ladies and gentlemen, what we are witnessing here is no ordinary feat. This courageous individual is embarking on a journey, a voyage into the realm of iron and sweat. Each lift, each curl, a Herculean effort defying the very laws of gravity itself. Behold the determination etched onto their visage, the pure concentration as muscles contract and expand in a beautiful dance of human strength and resilience. Oh, the thrill, the adventure, the raw power of the human spirit captured in this symphony of motion. Truly, an exhilarating spectacle to behold - the daily odyssey of the gym warrior!"


In [ ]:
PROMPT_MESSAGES = [
     {
            "role": "user",
            "content": f"Shorten this script so it can be read in about 17 seconds: {result.choices[0].message.content}",
        }
]
params = {
    "model": "gpt-4",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 500,
}

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)

"Ladies and gentlemen, witness this incredible journey! The power defying gravity with every lift, the determination and concentration. See the muscles dance, a testament of strength. The thrill, the spirit of our gym warrior conquering daily. Remarkable, breathtaking!"


## Text to Speech

In [ ]:
from elevenlabs import generate, play, set_api_key
set_api_key("")
audio = generate(
    text=result.choices[0].message.content,
    voice="Oliver - Documentary Narration",
    model='eleven_multilingual_v2'
)

play(audio)

## Add the TTS audio to the video clip

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
import os

video_clip = VideoFileClip("seated-biceps-curl.mp4")
audio_clip = AudioFileClip("Audio.mp3")

final_clip = video_clip.set_audio(audio_clip)

title = 'Exercise Narration'

final_clip.write_videofile(title + ".mp4")

Moviepy - Building video Exercise Narration.mp4.
MoviePy - Writing audio in Exercise NarrationTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video Exercise Narration.mp4



Moviepy - Done !
Moviepy - video ready Exercise Narration.mp4
